## Connect to your workspace

To get started, connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.34.0 to work with mm-aml-dev


In [2]:
import os
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

parent = os.path.dirname(directory_path)
parent_folder_name = os.path.basename(parent)
print("My user directory name is: " + parent_folder_name)

user = parent_folder_name
user = user.replace('_', '')
user = user.replace('-', '')
user = user[:10]
print('user=' + user)
experiment_name = parent_folder_name + '-003-code-a-thon-diabetes'

My current directory is : /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/code-a-thon
My directory name is : code-a-thon
My user directory name is: memasanz
user=memasanz


## Prepare data for an experiment

As you can see in the code above, it's easy to convert a tabular dataset to a Pandas dataframe, enabling you to work with the data using common python techniques.

In [3]:
from azureml.core import Dataset

#use default datastore
default_ds = ws.get_default_datastore()

if user + '-diabetes-tabular-dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.parquet'], # Upload the diabetes csv files in /data
                        target_path= user + '-diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)  
    parquet_paths = [(default_ds, user + '-diabetes-data/*.parquet')]
    tab_data_set  = Dataset.Tabular.from_parquet_files(path=parquet_paths)

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name= user + '-diabetes-tabular-dataset',
                                description='diabetes data',
                                tags = {'format':'parquet'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [4]:
# View Registered datasets in workspace
all_datasets = Dataset.get_all(ws)

for d in all_datasets:
    print(d)

diabetes-target
pipeline_diabetes_scored_data
pipeline_diabetes_inferencing_data
diabetes-tabular-dataset-raw
mm-synapse-base-NYC-20211103063643-dataset
memasanz_004_pipeline_diabetes_scored_data
memasanz_004_pipeline_diabetes_inferencing_data
memasanz-diabetes-tabular-dataset-raw
memasanz-diabetes-tabular-dataset
mm-synapse-base-nyc_taxi4-20211103071528-dataset
mm-synapse-base-NYC-20211103032007-dataset
test3
exp_Testing_Data
exp_Training_Data
exp_Raw_Data
demodata
demo
test
memasanztraining_registered_dataset
memasanz_prepped_dataset
registered_dataset
herego
inference-data-marathon-model-blob-3-marathon-aks-srv-insights-blob2
scored_data
inferencing_data
compressor_outlier_clip_train_y
compressor_outlier_clip_train_x
compressor_outlier_clip_train
compressor_outlier_clip
compressors_holdout_dataset
compressors_main_dataset
marathon_compressor_raw
inference-data-marathon-model-blob-2-marathon-aks-srv-insights-blob2
inference-data-marathon-model-blob-2-marathon-aks-srv-insights-blob
ed

### Create a file Dataset

The dataset you created is a *tabular* dataset that can be read as a dataframe containing all of the data in the structured files that are included in the dataset definition. This works well for tabular data, but in some machine learning scenarios you might need to work with data that is unstructured; or you may simply want to handle reading the data from files in your own code. To accomplish this, you can use a *file* dataset, which creates a list of file paths in a virtual mount point, which you can use to read the data in the files.

In [5]:
#Create a file dataset from the path on the datastore (this may take a short while)
file_data_set = Dataset.File.from_files(path=(default_ds, user + '-diabetes-data/*.parquet'))

# Get the files in the dataset
for file_path in file_data_set.to_path():
    print('dataset from file path in default storage: ' + file_path)
    
# Register the file dataset
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name= user + '-diabetes-file-dataset',
                                            description='example diabetes files',
                                            tags = {'format':'parquet'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

dataset from file path in default storage: /diabetes.parquet
Datasets registered


## Create a training script

Run the following two cells to create:

1. A folder for a new experiment
2. An training script file that uses **scikit-learn** to train a model and **matplotlib** to plot a ROC curve.

In [6]:
import os

# Create a folder for the experiment files
experiment_folder = user + '-003-compute-cluster-diabetes_code-a-thon-exp'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

memasanz-003-compute-cluster-diabetes_code-a-thon-exp folder created


In [7]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get script arguments
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# Set regularization hyperparameter
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing memasanz-003-compute-cluster-diabetes_code-a-thon-exp/diabetes_training.py


## Define an environment

When you run a Python script as an experiment in Azure Machine Learning, a Conda environment is created to define the execution context for the script. Azure Machine Learning provides a default environment that includes many common packages; including the **azureml-defaults** package that contains the libraries necessary for working with an experiment run, as well as popular packages like **pandas** and **numpy**.

You can also define your own environment in a Conda specification file, adding packages by using **conda** or **pip** to ensure your experiment has access to all the libraries it requires.

> **Note**: The conda dependencies are installed first, followed by the pip dependencies. Since the **pip** package is required to install the pip dependencies, it's good practice to include it in the conda dependencies.

Run the following cell to create a Conda specification file named *experiment_env.yml* in the same folder as this notebook.

In [8]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing memasanz-003-compute-cluster-diabetes_code-a-thon-exp/experiment_env.yml


Now you can use your custom conda environment file for your experiment

In [9]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification(user + "-experiment-env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

memasanz-experiment-env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



In [ ]:
import azureml.core.runconfig
from azureml.core import Environment, Experiment
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
diabetes_ds = ws.datasets.get(user + '-diabetes-tabular-dataset')

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env) 

# submit the experiment
experiment_name = user + '-003-code-a-thon-diabetes-local'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Create a compute cluster

In many cases, your local compute resources may not be sufficient to process a complex or long-running experiment that needs to process a large volume of data; and you may want to take advantage of the ability to dynamically create and use compute resources in the cloud. Azure Machine Learning supports a range of compute targets, which you can define in your workpace and use to run experiments; paying for the resources only when using them.

You can create a compute cluster in [Azure Machine Learning studio](https://ml.azure.com), or by using the Azure Machine Learning SDK. The following code cell checks your workspace for the existance of a compute cluster with a specified name, and if it doesn't exist, creates it.

> **Important**: Change *your-compute-cluster* to a suitable name for your compute cluster in the code below before running it - you can specify the name of an existing cluster if you have one. Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = user + "-cluster"
cluster_name = cluster_name[-16:]
print('trying to create: ' + cluster_name)

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2, idle_seconds_before_scaledown=1800)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

In [ ]:
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                                environment=experiment_env,
                                compute_target=cluster_name) # Use docker to host environment



#submit the experiment
experiment_name = user + '-003-code-a-thon-diabetes-remote'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()